Rahmanda Afebrio Yuris Soesatyo - Chapter 17:Representation Learning and Generative Learning Using Autoencoders and GANs

1. Undercomplete and Stacked Autoencoders

Autoencoder adalah model neural network yang terdiri dari encoder dan decoder, di mana encoder memampatkan input ke dalam representasi laten, sementara decoder bertugas merekonstruksi kembali input dari representasi tersebut. Pada undercomplete autoencoder, ukuran latent code dibuat lebih kecil dari dimensi input, sehingga model “dipaksa” mempelajari fitur-fitur paling penting agar rekonstruksi tetap mendekati data asli, biasanya dengan meminimalkan reconstruction loss seperti Mean Squared Error atau binary cross-entropy.

Jika digunakan aktivasi linear dan loss MSE, undercomplete autoencoder pada dasarnya akan berperilaku mirip dengan Principal Component Analysis (PCA), karena sama-sama mencari representasi berdimensi lebih rendah yang menangkap variansi utama data. Namun, keunggulan autoencoder muncul ketika arsitekturnya diperluas menjadi stacked autoencoder, yaitu encoder–decoder berlapis dengan struktur simetris, yang memungkinkan pemodelan hubungan nonlinier yang tidak dapat ditangkap oleh PCA.

Dalam praktik, stacked autoencoder sering digunakan untuk reduksi dimensi—misalnya sebagai tahap awal sebelum visualisasi data dengan t-SNE—serta untuk unsupervised pretraining. Pendekatan ini sangat berguna ketika data berlabel terbatas, karena model dapat terlebih dahulu belajar representasi umum dari data sebelum di-fine-tune lebih lanjut untuk tugas supervised seperti klasifikasi atau regresi.

In [ ]:
from tensorflow import keras

# Linear autoencoder 3D -> 2D (PCA-like)
encoder = keras.models.Sequential([
    keras.layers.Dense(2, input_shape=[3])        # no activation
])
decoder = keras.models.Sequential([
    keras.layers.Dense(3, input_shape=[2])        # no activation
])
autoencoder = keras.models.Sequential([encoder, decoder])
autoencoder.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=0.1))
history = autoencoder.fit(X_train, X_train, epochs=20)
codings = encoder.predict(X_train)

# Stacked autoencoder for Fashion MNIST
stacked_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(30, activation="selu"),
])
stacked_decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[30]),
    keras.layers.Dense(28 * 28, activation="sigmoid"),
    keras.layers.Reshape([28, 28]),
])
stacked_ae = keras.models.Sequential([stacked_encoder, stacked_decoder])
stacked_ae.compile(loss="binary_crossentropy",
                   optimizer=keras.optimizers.SGD(learning_rate=1.5))
history = stacked_ae.fit(X_train, X_train, epochs=10,
                         validation_data=(X_valid, X_valid))


2. Variants: Convolutional, Recurrent, Denoising, and Sparse Autoencoders

Berbagai varian autoencoder dikembangkan untuk menyesuaikan diri dengan karakteristik data yang berbeda-beda. Perbedaan ini terutama terletak pada jenis layer yang digunakan dan tujuan tambahan yang ingin dicapai selama proses pembelajaran representasi.

Convolutional autoencoder dirancang khusus untuk data citra dengan mengganti layer Dense menjadi convolution dan transposed convolution. Encoder menurunkan resolusi spasial sambil memperkaya jumlah channel, sedangkan decoder membalik proses tersebut untuk merekonstruksi gambar. Dengan cara ini, struktur spasial citra tetap terjaga dan jumlah parameter menjadi jauh lebih efisien dibanding autoencoder berbasis fully connected.

Untuk data sekuens seperti time series atau teks, recurrent autoencoder memanfaatkan RNN, LSTM, atau GRU. Encoder bertugas merangkum seluruh urutan menjadi satu representasi vektor, sementara decoder mengubah vektor tersebut kembali menjadi urutan. Biasanya, representasi laten diulang ke setiap langkah waktu menggunakan mekanisme seperti RepeatVector agar decoder dapat menghasilkan kembali sekuens input secara lengkap.

Denoising autoencoder dilatih dengan memberikan input yang telah ditambahkan noise, lalu meminta model merekonstruksi versi data yang bersih. Pendekatan ini mendorong model untuk belajar fitur yang lebih robust dan stabil terhadap gangguan, sehingga sering digunakan pada tugas image denoising sekaligus sebagai bentuk regularisasi representasi.

Sementara itu, sparse autoencoder menambahkan kendala sparsity pada aktivasi latent code, misalnya melalui regularisasi ℓ1 atau KL divergence terhadap tingkat aktivasi tertentu. Akibatnya, hanya sebagian kecil neuron yang aktif pada satu waktu, sehingga setiap neuron cenderung merepresentasikan fitur yang lebih spesifik dan bermakna. Secara keseluruhan, berbagai varian autoencoder ini menjadi fondasi penting dalam pembelajaran representasi tak terawasi dan banyak digunakan dalam pretraining, denoising, serta reduksi dimensi nonlinier.

In [ ]:

from tensorflow import keras

# Conv autoencoder (Fashion MNIST)
conv_encoder = keras.models.Sequential([
    keras.layers.Reshape([28, 28, 1], input_shape=[28, 28]),
    keras.layers.Conv2D(16, 3, padding="same", activation="selu"),
    keras.layers.MaxPool2D(2),
    keras.layers.Conv2D(32, 3, padding="same", activation="selu"),
    keras.layers.MaxPool2D(2),
    keras.layers.Conv2D(64, 3, padding="same", activation="selu"),
    keras.layers.MaxPool2D(2),
])
conv_decoder = keras.models.Sequential([
    keras.layers.Conv2DTranspose(32, 3, strides=2, padding="valid",
                                 activation="selu", input_shape=[3, 3, 64]),
    keras.layers.Conv2DTranspose(16, 3, strides=2, padding="same",
                                 activation="selu"),
    keras.layers.Conv2DTranspose(1, 3, strides=2, padding="same",
                                 activation="sigmoid"),
    keras.layers.Reshape([28, 28]),
])
conv_ae = keras.models.Sequential([conv_encoder, conv_decoder])

# Recurrent autoencoder (treat image as 28×(sequence of 28 pixels))
recurrent_encoder = keras.models.Sequential([
    keras.layers.LSTM(100, return_sequences=True, input_shape=[None, 28]),
    keras.layers.LSTM(30),
])
recurrent_decoder = keras.models.Sequential([
    keras.layers.RepeatVector(28, input_shape=[30]),
    keras.layers.LSTM(100, return_sequences=True),
    keras.layers.TimeDistributed(
        keras.layers.Dense(28, activation="sigmoid")
    ),
])
recurrent_ae = keras.models.Sequential([recurrent_encoder, recurrent_decoder])

# Denoising AE via Dropout
dropout_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(30, activation="selu"),
])
dropout_decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[30]),
    keras.layers.Dense(28 * 28, activation="sigmoid"),
    keras.layers.Reshape([28, 28]),
])
dropout_ae = keras.models.Sequential([dropout_encoder, dropout_decoder])

# Sparse AE with L1 activity regularization
sparse_l1_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(300, activation="sigmoid"),
    keras.layers.ActivityRegularization(l1=1e-3),
])
sparse_l1_decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[300]),
    keras.layers.Dense(28 * 28, activation="sigmoid"),
    keras.layers.Reshape([28, 28]),
])
sparse_l1_ae = keras.models.Sequential([sparse_l1_encoder, sparse_l1_decoder])


3. Sparse Autoencoders with KL Divergence and Variational Autoencoders


Pendekatan sparsity yang lebih fleksibel dapat dicapai dengan menambahkan KL divergence sebagai regularisasi antara tingkat sparsity target ( p ) dan rata-rata aktivasi aktual ( q ) dari setiap neuron pada latent code. Regularizer ini mendorong agar nilai aktivasi rata-rata tiap neuron mendekati ( p ) (misalnya 0.05–0.1), sehingga sebagian besar neuron tetap tidak aktif dan hanya merespons ketika fitur tertentu muncul. Dibandingkan regularisasi ℓ1 sederhana, metode ini memberikan kontrol yang lebih eksplisit terhadap distribusi aktivasi neuron.

Variational Autoencoder (VAE) memperluas autoencoder deterministik menjadi model probabilistik dan generatif. Alih-alih memetakan input ke satu titik laten, encoder menghasilkan parameter distribusi laten berupa mean ( \mu ) dan log-variance ( \log \sigma^2 ). Dengan menggunakan reparameterization trick, sampling dari distribusi laten dapat dilakukan secara diferensiabel, memungkinkan backpropagation tetap berjalan.

Fungsi loss VAE terdiri dari dua komponen utama:

reconstruction loss, yang memastikan output tetap menyerupai input,
latent loss, berupa KL divergence antara distribusi laten hasil encoder dan distribusi Gaussian standar.
Kombinasi ini memaksa latent space menjadi halus dan terstruktur, sehingga memungkinkan interpolasi bermakna antar titik laten dan generasi sampel baru yang realistis.

In [ ]:
from tensorflow import keras
import tensorflow.keras.backend as K
import tensorflow as tf

# KL-based sparsity regularizer
kl_divergence = keras.losses.kullback_leibler_divergence

class KLDivergenceRegularizer(keras.regularizers.Regularizer):
    def __init__(self, weight, target=0.1):
        self.weight = weight
        self.target = target

    def __call__(self, inputs):
        mean_activities = K.mean(inputs, axis=0)
        return self.weight * (
            kl_divergence(self.target, mean_activities) +
            kl_divergence(1. - self.target, 1. - mean_activities)
        )

kld_reg = KLDivergenceRegularizer(weight=0.05, target=0.1)
sparse_kl_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(300, activation="sigmoid",
                       activity_regularizer=kld_reg),
])
sparse_kl_decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[300]),
    keras.layers.Dense(28 * 28, activation="sigmoid"),
    keras.layers.Reshape([28, 28]),
])
sparse_kl_ae = keras.models.Sequential([sparse_kl_encoder, sparse_kl_decoder])

# VAE components
class Sampling(keras.layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs
        eps = K.random_normal(tf.shape(log_var))
        return eps * K.exp(log_var / 2) + mean

codings_size = 10
inputs = keras.layers.Input(shape=[28, 28])
z = keras.layers.Flatten()(inputs)
z = keras.layers.Dense(150, activation="selu")(z)
z = keras.layers.Dense(100, activation="selu")(z)
codings_mean = keras.layers.Dense(codings_size)(z)
codings_log_var = keras.layers.Dense(codings_size)(z)
codings = Sampling()([codings_mean, codings_log_var])
variational_encoder = keras.Model(
    inputs=[inputs],
    outputs=[codings_mean, codings_log_var, codings]
)

decoder_inputs = keras.layers.Input(shape=[codings_size])
x = keras.layers.Dense(100, activation="selu")(decoder_inputs)
x = keras.layers.Dense(150, activation="selu")(x)
x = keras.layers.Dense(28 * 28, activation="sigmoid")(x)
outputs = keras.layers.Reshape([28, 28])(x)
variational_decoder = keras.Model(
    inputs=[decoder_inputs], outputs=[outputs]
)

_, _, codings = variational_encoder(inputs)
reconstructions = variational_decoder(codings)
variational_ae = keras.Model(inputs=[inputs], outputs=[reconstructions])

latent_loss = -0.5 * K.sum(
    1 + codings_log_var - K.exp(codings_log_var) - K.square(codings_mean),
    axis=-1
)
variational_ae.add_loss(K.mean(latent_loss) / 784.)
variational_ae.compile(loss="binary_crossentropy", optimizer="rmsprop")

history = variational_ae.fit(X_train, X_train, epochs=50, batch_size=128,
                             validation_data=(X_valid, X_valid))

# Sampling new images
codings_new = tf.random.normal(shape=[12, codings_size])
images = variational_decoder(codings_new).numpy()

4. Basics of GAN and Custom Training Loop

Generative Adversarial Network (GAN) terdiri dari dua model yang dilatih secara adversarial: generator, yang mengubah noise acak (biasanya Gaussian) menjadi data sintetis, dan discriminator, yang bertugas membedakan data asli dari data palsu. Proses training dilakukan secara bergantian dalam dua tahap pada setiap iterasi.

Pertama, discriminator dilatih menggunakan batch gabungan data asli dan data palsu, dengan label masing-masing “real” dan “fake”. Kedua, generator dilatih melalui model gabungan (GAN) dengan tujuan menipu discriminator, yaitu membuat discriminator mengklasifikasikan output generator sebagai data asli. Pada tahap ini, bobot discriminator dibekukan sehingga gradien yang mengalir hanya memperbarui generator.

Generator tidak pernah mengakses data asli secara langsung, melainkan hanya menerima sinyal gradien dari discriminator. Jika discriminator cukup kuat, gradien tersebut secara implisit mengandung informasi tentang struktur distribusi data asli. Namun, pelatihan GAN terkenal sulit dan rentan terhadap berbagai masalah, seperti:

mode collapse, di mana generator hanya menghasilkan variasi terbatas dari data,
ketidakstabilan training, dengan osilasi atau divergensi loss,
sensitivitas tinggi terhadap hyperparameter dan arsitektur.
Karena itu, GAN sering dilatih menggunakan custom training loop dan berbagai modifikasi (misalnya Wasserstein loss, gradient penalty, atau spectral normalization) untuk meningkatkan stabilitas dan kualitas hasil generatif.

In [ ]:
from tensorflow import keras
import tensorflow as tf

codings_size = 30
generator = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[codings_size]),
    keras.layers.Dense(150, activation="selu"),
    keras.layers.Dense(28 * 28, activation="sigmoid"),
    keras.layers.Reshape([28, 28]),
])

discriminator = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(150, activation="selu"),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(1, activation="sigmoid"),
])

gan = keras.models.Sequential([generator, discriminator])

discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

def train_gan(gan, dataset, batch_size, codings_size, n_epochs=50):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
        for X_batch in dataset:
            # Phase 1: train discriminator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            generated_images = generator(noise)
            X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)

            # Phase 2: train generator (through GAN)
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)

train_gan(gan, dataset, batch_size, codings_size)


5. DCGAN and Practical GAN Tricks

DCGAN (Deep Convolutional GAN) mengusulkan seperangkat prinsip arsitektur yang terbukti meningkatkan stabilitas pelatihan GAN pada data gambar. Inti pendekatannya adalah mengganti operasi pooling dengan convolution dan transposed convolution ber-stride untuk melakukan downsampling dan upsampling secara terpelajar. Hampir semua layer dilengkapi Batch Normalization untuk menstabilkan distribusi aktivasi, sementara fully connected hidden layer dihilangkan agar model bersifat sepenuhnya konvolusional.

Pada sisi generator, fungsi aktivasi ReLU (atau variasinya) digunakan di hampir seluruh layer, dengan tanh di layer output. Generator menerima vektor laten berdimensi tetap (misalnya 100), memproyeksikannya menjadi feature map kecil (misalnya 7×7×128), lalu melakukan upsampling bertahap hingga mencapai resolusi target, seperti 28×28 untuk MNIST. Sebaliknya, discriminator menggunakan leaky ReLU untuk menjaga aliran gradien dan bertindak sebagai CNN klasifikasi biner yang secara bertahap menurunkan resolusi spasial.

Agar konsisten dengan output tanh, citra input ke discriminator perlu di-reshape ke format [batch, height, width, channels] dan dinormalisasi ke rentang ([-1, 1]). Dropout sering ditambahkan pada discriminator sebagai regularisasi tambahan untuk mencegah overfitting terhadap data training.

In [ ]:
from tensorflow import keras

codings_size = 100
generator = keras.models.Sequential([
    keras.layers.Dense(7 * 7 * 128, input_shape=[codings_size]),
    keras.layers.Reshape([7, 7, 128]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(
        64, kernel_size=5, strides=2, padding="same", activation="selu"
    ),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(
        1, kernel_size=5, strides=2, padding="same", activation="tanh"
    ),
])

discriminator = keras.models.Sequential([
    keras.layers.Conv2D(
        64, kernel_size=5, strides=2, padding="same",
        activation=keras.layers.LeakyReLU(0.2),
        input_shape=[28, 28, 1],
    ),
    keras.layers.Dropout(0.4),
    keras.layers.Conv2D(
        128, kernel_size=5, strides=2, padding="same",
        activation=keras.layers.LeakyReLU(0.2),
    ),
    keras.layers.Dropout(0.4),
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation="sigmoid"),
])

gan = keras.models.Sequential([generator, discriminator])

# Prepare data: reshape & rescale to [-1, 1]
X_train_dcgan = X_train.reshape(-1, 28, 28, 1) * 2. - 1.

discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

# Training loop sama seperti fungsi train_gan di atas, hanya mengganti X_train_dcgan


6. Advanced GANs: Progressive Growing, StyleGAN, and Training Tricks

Untuk menghasilkan gambar beresolusi tinggi secara stabil, diperkenalkan teknik Progressive Growing of GANs, di mana pelatihan dimulai dari resolusi sangat kecil (misalnya 4×4) dan secara bertahap ditingkatkan ke resolusi yang lebih besar (8×8, 16×16, dan seterusnya). Layer baru ditambahkan secara perlahan menggunakan mekanisme fade-in, yaitu interpolasi antara output jaringan lama dan jaringan yang diperluas dengan koefisien ( \alpha ), sehingga transisi resolusi berlangsung mulus dan stabil.

Beberapa teknik tambahan turut digunakan untuk meningkatkan stabilitas dan keragaman sampel, seperti minibatch standard deviation layer untuk mendorong variasi antar sampel, equalized learning rate untuk menyeimbangkan skala bobot antar layer, serta pixelwise normalization pada generator.

StyleGAN melangkah lebih jauh dengan memisahkan latent code awal ( z ) dari representasi gaya ( w ) melalui sebuah mapping network berbasis MLP. Representasi gaya ini kemudian disuntikkan ke setiap level resolusi dalam synthesis network menggunakan Adaptive Instance Normalization (AdaIN), memungkinkan kontrol terpisah terhadap atribut global dan lokal gambar. Selain itu, noise acak eksplisit ditambahkan pada tiap level untuk memodelkan detail stokastik seperti tekstur kulit atau helai rambut.

Teknik style mixing menggunakan lebih dari satu latent code pada level resolusi yang berbeda, sehingga jaringan tidak mempelajari ketergantungan semu antar level dan terdorong untuk melokalisasi fitur secara semantik. Pendekatan-pendekatan ini menghasilkan gambar wajah dengan kualitas sangat tinggi, memungkinkan interpolasi laten yang halus dan manipulasi semantik seperti perubahan ekspresi, usia, atau atribut visual tertentu.

Bab ini juga menyinggung berbagai pengembangan lain, termasuk conditional GAN, experience replay, dan mini-batch discrimination, serta pentingnya evaluasi kualitas dan diversitas hasil generatif. Meskipun kemajuannya signifikan, pelatihan GAN tetap merupakan area riset yang menantang dan terus berkembang.